# Aula 3 - Medidas de Centralidade

### Importando as bibliotecas

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Função que calcula o Brokering

In [2]:
def BrokeringCentrality(G, normalized=True):
    '''
    Computes Brokering Centrality for a graph G.
    For each node brokering centrality is defined as:
        brokering[node] =  (1 - clustering[node]) * degree[node]

    Parameters
    ----------
        G:  networkx graph
            Graph to be computed brokering centrality.

    Returns
    -------
        brokering: dictionary
                   Dictionary of nodes with brokering centrality as the value.
    '''
    Degree     = nx.degree_centrality(G)
    Clustering = nx.clustering(G)
    Brokering  = {}
    for node in G.nodes():
        Brokering[node] =  (1 - Clustering[node]) * Degree[node]
    #
    if normalized == True:
        NormalizeDict(Brokering)
    #
    return Brokering

### Função que calcula o Bridgeness

In [3]:
def BridgenessCentrality(G, Betweenness=None, normalized=True):
    '''
    Computes Bridgeness Centrality for a graph G.

    For each node the bridgeness coefficient is defined as:
        bridge_coeff[node] =  (1/degree[node]) / sum(1/degree[neighbors[node]])

    The bridgeness centrality of a node is defined as:
        Bridgeness[node] = Betweenness(node) * bridge_coeff[node]

    Parameters
    ----------
        G:  networkx graph
            Graph to be computed brokering centrality.
        Betweenness:    dictionary
            Dictionary of nodes with betweenness centrality as value.

            Obs: since computing of betweennes can take a lot of time,
            it's possible to provide the betweenness as a parameter,
            if it was already computed before, to avoid compute it
            multiple times.

    Returns
    -------
        Bridgeness: dictionary
                    Dictionary of nodes with bridgeness centrality as value.

    Note: Only nodes with degree >= 1 will be returned.

    '''
    if Betweenness is None:
        Betweenness = nx.betweenness_centrality(G)
    #
    Bridgeness = {}
    for node in G.nodes():
        degree_node = nx.degree(G,node)
        if degree_node > 0:
            # sum(1/degree[neighbors[node]]
            neighbors_degree  = dict(nx.degree(G, nx.neighbors(G, node))).values()
            sum_neigh_inv_deg = sum((1.0/d) for d in neighbors_degree)
            #
            bridge_coeff = (1.0/degree_node) / sum_neigh_inv_deg
            #
            Bridgeness[node] = Betweenness[node] * bridge_coeff
    #
    if normalized == True:
        NormalizeDict(Bridgeness)
    #
    return Bridgeness

### Função que normaliza o dicionário de valores

In [4]:
def NormalizeDict(D):
    '''
    Normalizes a dictionary of values.
    Finds the maximum value of a dictionary and divides all values by it.

    Parameters
    ----------
        D:  dictionary
            Dictionary of float numbers as values.

    Returns
    -------
        ND: dictionary
            Normalized (in-place) Dictionary with float number as value.
    '''
    max_d = max(D.values())
    for key in D:
        D[key] /= max_d

### Função que calcula as medidas de centralidade

In [5]:
def ComputeCentralityMeasures(G, normalized=False):
    '''
    Computes centrality measures of a graph G.

    Parameters
    ----------
        G:  networkx graph
            Graph to be computed brokering centrality.
            G must be in one single component.

    Returns
    -------
        Centrality Measures Dictionaries:
            Degree, Betweenness, Closeness,
            Clustering, Brokering, Bridgeness
    '''
    #Degree      = nx.degree_centrality(G)                             #  0s
    Degree      = nx.degree(G)                                         #  0s
    Clustering  = nx.clustering(G)                                     #  4s
    Betweenness = nx.betweenness_centrality(G,normalized=normalized)   #  ~2750s
    Closeness   = nx.closeness_centrality(G)                           #  ~800s
    
    Brokering   = BrokeringCentrality(G,normalized=normalized)         #  ~8s
    Bridgeness  = BridgenessCentrality(G,Betweenness=Betweenness,normalized=normalized)
    #
    # replace code below to use normalize on Pandas
    if normalized == True:
        NormalizeDict(Degree)
        NormalizeDict(Betweenness)
        NormalizeDict(Closeness)
        NormalizeDict(Clustering)
        NormalizeDict(Brokering)
        NormalizeDict(Bridgeness)
    #
    return (Degree, Betweenness, Closeness, Clustering, Brokering, Bridgeness)

### Função que escreve as medidas em um arquivo

In [6]:
def WriteCentralityMeasures(filename, NodesOrder, Degree, Betweenness,
                            Closeness, Clustering, Brokering, Bridgeness):
    '''
    Writes, in a file, the centrality measures of a graph G.

    Parameters
    ----------
        G:  networkx graph
            Graph to be computed brokering centrality.
            G must be in one single component.
    '''
    f = open(filename, 'w+')
    f.write('GENE\tDegree\tBetweenness\tCloseness' +\
              '\tClustering\tBrokering\tBridgeness\n')
    for g in NodesOrder:
        line = '%s\t%.15f\t%.15f\t%.15f\t%.15f\t%.15f\t%.15f\n' % (
                g, Degree[g], Betweenness[g], Closeness[g],
                Clustering[g], Brokering[g], Bridgeness[g])
        f.write(line)
    f.close()

### Função que transforma um dataframe para um grafo

In [7]:
def DataFrame2Graph(dataframe):
    G = nx.Graph()
    for i in range(dataframe.shape[0]):
        # retrieve the first and the second column for each line
        u,v = dataframe.iloc[i]
        # adds the edge (u,v) to the graph 
        G.add_edge(u,v)
    #
    return G

## Exemplo de Uso

### Carrega os arquivos e insere em um DataFrame

In [8]:
df1 = pd.read_csv('1a-greenyellow_case.txt', sep="\t")
df2 = pd.read_csv('1b-greenyellow_control.txt', sep="\t")

### Transforma os DataFrames carregados em grafos

In [9]:
G1 = DataFrame2Graph(df1)
G2 = DataFrame2Graph(df2)

### Exemplo de como realizar a chamadas de função para uma medida específica

In [10]:
Degree      = nx.degree(G1)

In [11]:
Clustering  = nx.clustering(G1)                                     

In [12]:
Betweenness = nx.betweenness_centrality(G1,normalized=False)

In [13]:
Closeness   = nx.closeness_centrality(G1)                

In [14]:
Brokering   = BrokeringCentrality(G1,normalized=False)         

In [15]:
Bridgeness  = BridgenessCentrality(G1,Betweenness=Betweenness,normalized=False)

### Retorna o resultado do cálculo para cada medida de centralidade

#### Resultado para o G1

In [16]:
Degree, Betweenness, Closeness, Clustering, \
Brokering, Bridgeness = ComputeCentralityMeasures(G1, normalized=False)

#### Resultado para o G2

In [17]:
Degree, Betweenness, Closeness, Clustering, \
Brokering, Bridgeness = ComputeCentralityMeasures(G2, normalized=False)